In [ ]:
# default_exp util

In [ ]:
# export
import IPython.display

In [ ]:
# export
def render_latex(formula: str) -> str:
    
    assert formula[0] == '$'
    
    return IPython.display.Markdown(r'$\Large' + formula[1:])

A decorator

In [ ]:
# export
def to_formula_maybe(func):
    
    def wrapper(*args, **kwargs):
        
        if ('to_formula' in kwargs) and (kwargs['to_formula']):
            
            kwargs.pop('to_formula')
            
            return f'${func(*args, **kwargs)}$'
        
        else:
            
            return func(*args, **kwargs)
    return wrapper